# Dataset augmentation

## Rotation
* Yaw direction: 0 ~ 342, -- 20 steps
* Pitch direction: -1, 0, 1, -- 3 steps

![example](../image/aug_rot_example.png)

## Rotation - Negative sample
* Yaw direction: 0, 90, -- 2 steps
* Pitch direction: -1, 0, -- 2 steps

![example](../image/ndb_aug.png)

In [8]:
""" All data processing """

import os

import pickle

from utils import *

from tqdm import trange, tqdm

""" Load dataset """
""" v1 """
# db_path = '/media/kyungpyo/SmartCarContest/MasterThesisProject/Webots/dataset/db_v1_20181019.pickle'

""" Load dataset """
""" v2 """
# db_path = '/media/kyungpyo/SmartCarContest/MasterThesisProject/Webots/dataset/db_v2_20181019.pickle'

""" Load dataset """
""" v4 """
db_path = os.path.abspath('../dataset/db_v4_20181031.pickle')


object_list = []
object_list.append('world_RangeRoverSportSVRSimple')
object_list.append('world_BmwX5Simple')
object_list.append('world_CitroenCZeroSimple')
object_list.append('world_LincolnMKZSimple')
object_list.append('world_ToyotaPriusSimple')
object_list.append('world_BusSimple')
object_list.append('world_TruckSimple')
object_list.append('world_Pedestrian')
object_list.append('world_MotorbikeSimple')
object_list.append('world_ScooterSimple')


## Load pickle
with open(db_path, 'rb') as f:
    db = pickle.load(f)    

db_aug = {}

for idx in range( len(object_list) ):
    
    cloud_list = db[object_list[idx]]
    db_aug[object_list[idx]] = []
    
    print (object_list[idx])
    sys.stdout.flush()
    
    for cloud in tqdm(cloud_list):
        """ Augmentation - rotation (yaw) """
        yaw_rotate_from = 0.
        yaw_rotate_to = 360.
        yaw_step = 20.

        pitch_rotate_from = -1.
        pitch_rotate_to = 1.
        pitch_step = 3.

        # numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)[source]

        yaw_iter = np.linspace(yaw_rotate_from, yaw_rotate_to, yaw_step, endpoint = False)
        pitch_iter = np.linspace(pitch_rotate_from, pitch_rotate_to, pitch_step)

        cloud_aug = []
        
        i = 0
        
        for yaw in yaw_iter:

            yaw = yaw * np.pi / 180.0

            for pitch in pitch_iter:                       
                i += 1
                
                pitch = pitch * np.pi / 180.0

                yawMatrix = np.matrix([[np.cos(yaw), -np.sin(yaw), 0.],
                                    [np.sin(yaw), np.cos(yaw), 0.],
                                    [0., 0., 1.]])

                pitchMatrix = np.matrix([[np.cos(pitch), 0., np.sin(pitch)],
                                        [0., 1., 0.],
                                        [-np.sin(pitch), 0., np.cos(pitch)]])

                p_mean = np.mean(cloud, axis = 0)

                cloud[:,0] -= p_mean[0]
                cloud[:,1] -= p_mean[1]
                cloud[:,2] -= p_mean[2]

                np_pc_aug = np.array((pitchMatrix*yawMatrix*cloud.T).T)                                                    
                db_aug[object_list[idx]].append(np_pc_aug)      

world_RangeRoverSportSVRSimple


  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
100%|██████████| 8/8 [00:00<00:00, 128.77it/s]

world_BmwX5Simple



100%|██████████| 8/8 [00:00<00:00, 123.57it/s]

world_CitroenCZeroSimple


...


100%|██████████| 8/8 [00:00<00:00, 120.84it/s]

world_MotorbikeSimple



100%|██████████| 8/8 [00:00<00:00, 126.47it/s]

world_ScooterSimple



100%|██████████| 8/8 [00:00<00:00, 125.20it/s]


In [9]:
for key in object_list:
    print key, len(db_aug[key])

world_RangeRoverSportSVRSimple 480
world_BmwX5Simple 480
world_CitroenCZeroSimple 480
world_LincolnMKZSimple 480
world_ToyotaPriusSimple 480
world_BusSimple 480
world_TruckSimple 480
world_Pedestrian 480
world_MotorbikeSimple 480
world_ScooterSimple 480


In [10]:
## Save pickle
with open('db_aug_v4_20181031.pickle', 'wb') as f:
    pickle.dump(db_aug, f)

In [15]:
## Load pickle
data_path = os.path.abspath('../dataset/db_aug_v4_20181031.pickle')
with open(data_path, 'rb') as f:
    db_aug_test = pickle.load(f)   

""" Display augmenation results - yaw direction """
%matplotlib qt

cloud_aug = db_aug_test[object_list[-1]][:60]

plt.close('all')
fig = plt.figure()

fig.subplots_adjust(left=0.02,top= 0.98,bottom=0.02,right=0.98,wspace=0.1,hspace=0.5)

ax = fig.add_subplot(881, projection='3d')
ax = display_point_cloud_box_ax(ax, cloud_aug[1])
ax.set_title('Original data')

for i,pc in enumerate(cloud_aug):
    ctr=i+2
    ax=fig.add_subplot(8,8,ctr, projection='3d')
    ax = display_point_cloud_box_ax(ax, pc) 
    ax.axis('off')

plt.show()

In [16]:
## Load pickle
with open('../dataset/ndb_aug_v2_20181020.pickle', 'rb') as f:
    db_aug_test = pickle.load(f)   

""" Display augmenation results - yaw direction """
%matplotlib qt

cloud_aug = db_aug_test['unknown'][:4]

plt.close('all')
fig = plt.figure()

fig.subplots_adjust(left=0.02,top= 0.98,bottom=0.02,right=0.98,wspace=0.1,hspace=0.5)

ax = fig.add_subplot(231, projection='3d')
ax = display_point_cloud_box_ax(ax, cloud_aug[1])
ax.set_title('Original data')

for i,pc in enumerate(cloud_aug):
    ctr=i+2
    ax=fig.add_subplot(2,3,ctr, projection='3d')
    ax = display_point_cloud_box_ax(ax, pc) 
    ax.axis('off')
    
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
plt.show()